In [1]:
using Flux
using Flux.Data: DataLoader
using Statistics: mean
using BSON
include("preprocessing.jl")

reconstruct_midi_file (generic function with 1 method)

In [2]:
function prepare_data(input_dir::String)
    inputs = []
    outputs = []

    for csv in readdir(input_dir)
        df = CSV.read(joinpath(input_dir, csv), DataFrame)
        if size(df)[1] < 500
            continue
        end
        push!(inputs, Matrix{Float32}(df[1:500, 1:end-1]))
        push!(outputs, sum(df[1:500, end]))
    end
    
    # Find the length of the longest input array
    max_length = maximum(size(input, 1) for input in inputs)

    # Pad input arrays with zeros to match the longest array's length
    padded_inputs = []
    for input in inputs
        rows_to_pad = max_length - size(input, 1)
        padded_input = vcat(input, zeros(Float32, rows_to_pad, size(input, 2)))
        push!(padded_inputs, padded_input)
    end

    (padded_inputs, outputs)
end

prepare_data (generic function with 1 method)

In [3]:
# Hyperparameters
hidden_size = 128
output_size = 1
epochs = 25
batch_size = 32
learning_rate = 0.001


0.001

In [4]:
inputs, targets = prepare_data("assets/anomalous")

trainRange = 1:Int(floor(0.8 * length(inputs)))
valRange = Int(floor(0.8 * length(inputs))):length(inputs)

train_data = inputs[trainRange], targets[trainRange]
val_data = inputs[valRange], targets[valRange]

(Any[Float32[68.0 40.0 5280.0 1110.0; 65.0 34.0 5052.0 480.0; … ; 54.0 38.0 126720.0 1440.0; 44.0 51.0 126555.0 360.0], Float32[56.0 65.0 1.0 480.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[43.0 27.0 1.0 1156.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[54.0 65.0 0.0 582.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[56.0 65.0 1.0 480.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[56.0 65.0 1.0 480.0; 60.0 77.0 0.0 1430.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[56.0 65.0 1.0 480.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[56.0 65.0 1.0 480.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[52.0 65.0 1.0 662.0; 68.0 77.0 1.0 480.0; … ; 75.0 45.0 73800.0 60.0; 70.0 45.0 73920.0 480.0], Float32[56.0 65.0 1.0 480.0; 68.0 97.0 1.0 108

In [5]:
# inputs, outputs = prepare_data("assets/anomalous")

rows = [size(arr, 1)[1] for arr in inputs]
numCSVs = size(inputs)[1]
SEQ_MIN = minimum(rows)
SEQ_MAX = maximum(rows)

println("Number of CSVs: $numCSVs " * "Min Rows: $SEQ_MIN " * "Max Rows: $SEQ_MAX " * "Number of columns: $(size(inputs[1])[2])")

Number of CSVs: 8550 Min Rows: 500 Max Rows: 500 Number of columns: 4


In [17]:
# Define the RNN model
model = Flux.Chain(
    Flux.RNN(500, 128, tanh),
    Flux.LSTM(128, 64),
    Flux.Dense(64, 1),
    Flux.relu
)

Chain(
  Recur(
    RNNCell(500 => 128, tanh),          # 80_640 parameters
  ),
  Recur(
    LSTMCell(128 => 64),                # 49_536 parameters
  ),
  Dense(64 => 1),                       # 65 parameters
  NNlib.relu,
)         # Total: 11 trainable arrays, 130_241 parameters,
          # plus 3 non-trainable, 256 parameters, summarysize 509.457 KiB.

In [16]:
function loss_gradient(model, x, y)
    loss = Flux.Losses.mse
    loss_val = loss(model(x), y)
    grad = Flux.gradient(() -> loss(model(x), y), Flux.params(model))
    return loss_val, grad
end

opt = Flux.Optimise.Adam(learning_rate)
opt = Flux.setup(opt, model)

(layers = ((cell = (σ = (), Wi = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), Wh = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), b = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999))), state0 = Leaf(Adam{Float64}(0.001, (0.9, 0.999), 1.0e-8), (Float32[0.0; 0.0; … ; 0.0; 0.0;;], Float32[0.0; 0.0; … ; 0.0; 0.0;;], (0.9, 0.999)))), state = ()), (cell = (Wi = Leaf(Ad

In [8]:
create_batches(r, xs, ys) = (Flux.batch(xs[r]), Flux.batch(ys[r]))

create_batches (generic function with 1 method)

In [18]:
function train_model(model, train_data, val_data, epochs, opt)
    train_inputs, train_outputs = train_data
    val_inputs, val_outputs = val_data
    train_losses = []
    val_losses = []
    best_val_loss = Inf

    for epoch in 1:epochs
        train_loss = 0
        val_loss = 0
        num_batches = ceil(length(train_inputs) / batch_size)

        # Train the model
        for i in 1:num_batches
            r = Int((i-1)*batch_size+1) : Int(min(i*batch_size, length(train_inputs)))
            println(r)
            x, y = create_batches(r, train_inputs, train_outputs)
            loss_val, grad = loss_gradient(model, x, y)
            update!(opt, Flux.params(model), grad)
            train_loss += loss_val
        end

        # Compute validation loss
        for i in 1:ceil(length(val_inputs) / batch_size)
            r = (i-1)*batch_size+1 : min(i*batch_size, length(val_inputs))
            x, y = create_batches(r, val_inputs, val_outputs)
            val_loss += Flux.mae(model(x), y)
        end

        train_loss /= num_batches
        val_loss /= ceil(length(val_inputs) / batch_size)
        push!(train_losses, train_loss)
        push!(val_losses, val_loss)

        # Print progress
        println("Epoch $(epoch) - Train loss: $(train_loss), Val loss: $(val_loss)")

        # Check if the current model is the best so far
        if val_loss < best_val_loss
            best_val_loss = val_loss
            best_model_params = deepcopy(Flux.params(model))
        end
    end

    # Restore the best model parameters
    Flux.loadparams!(model, best_model_params)

    # Return the trained model and the loss history
    return model, train_losses, val_losses
end


train_model (generic function with 1 method)

In [19]:
train_model(model, train_data, val_data, epochs, opt)

1:32


LoadError: DimensionMismatch: loss function expects size(ŷ) = (1, 4, 32) to match size(y) = (32,)

In [ ]:
# Test the model on the validation set
val_predictions = [model(reshape(x, size(x, 1), 1, size(x, 2))) for x in X_val]
val_loss = mean(loss_function(ŷ, y) for (ŷ, y) in zip(val_predictions, y_val))
println("Validation Loss: $val_loss")

In [ ]:
function preprocess_new_midi(csv_file::String)
    df = CSV.read(csv_file, DataFrame)
    if size(df)[1] < 500
        println("MIDI file is too short (< 500 rows).")
        return nothing
    end
    input = Matrix{Float32}(df[1:500,Not(:anomalies)])
    if sum(df.anomalies[1:500]) > 0
        println("MIDI file contains $(sum(df.anomalies[1:500])) anomalies.")
    end
    return input
end


In [ ]:
display(model)

In [ ]:
@save "initModel.bson" model

In [ ]:
model = BSON.@load model_filename model

In [ ]:
display(model)

In [ ]:
# Preprocess the new MIDI file
new_midi_file = "assets/anomalous/liz_rhap15_0.3.csv"
new_midi_input = preprocess_new_midi(new_midi_file)

# Check if the preprocessing was successful (the file had at least 500 rows)
if new_midi_input !== nothing
    # Reshape the input array to match the model's input shape
    new_midi_input = reshape(new_midi_input, size(new_midi_input, 1), 1, size(new_midi_input, 2))
    println(new_midi_input)    
    # Predict the number of errors in the new MIDI file
    num_errors = model(new_midi_input)

    println("Predicted number of errors: ", num_errors[1])
else
    println("Prediction cannot be performed due to insufficient data.")
end